# Decision theory and Bayesian methods: loss functions

Source:
- C. Davidson-Pilon, "Bayesian methods for hackers", Addison-Wesley (2016). A version updated to PyMC3 is available [here](https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/) and this notebook follows part of Chapter 5.

In [ ]:
import numpy as np
import pymc3 as pm
from scipy.optimize import fmin
from sklearn.linear_model import LinearRegression
from tqdm import tqdm_notebook as tqdm
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode(connected=True)

## Define a loss function

Define a loss function that is asymmetric in the sign of the error between the true return and the predicted one: we want to penalise (big loss) the cases in which we are predicting a return that has a different sign from the true one. This will result in more conservative choices when we are predicting a gain when we'd in fact have a loss and when we are predicting a loss when we'd in fact have a gain.

The function is
$$
L(R, \hat{y}) \equiv \left\lbrace
\begin{array}{l}
\alpha\hat{y}^2 - \text{sgn}(R)\,\hat{y} + |R|,\, \text{if}\, R \hat{y}<0\\
|R - \hat{y}|,\, \text{otherwise}
\end{array}
\right.
$$

where $R$ is the true return and $\hat{y}$ is the predicted one and $\alpha$ is an arbitrary (big) coefficient. This function grows linearly with the error when the prediction and the true return have the same sign, but in the opposite case grows quadratically _with the prediction_ (but not necessarily with the error $(R - \hat{y})$).

In [ ]:
def stock_loss(true_return, prediction, alpha=100.):
    """
    Loss function, asymmetric between the case in which prediction
    and true value have the same sign and the one in which the sign
    is opposite.
    """
    if true_return*prediction < 0:
        return alpha*prediction**2 -np.sign(true_return)*prediction + np.abs(true_return)
    else:
        return np.abs(true_return - prediction)

In [ ]:
trace_pos = go.Scatter(
    x=np.linspace(-0.1, 0.1, 100),
    y=[stock_loss(0.05, pred) for pred in np.linspace(-0.1, 0.1, 100)],
    name='True value 0.05'
)

trace_neg = go.Scatter(
    x=np.linspace(-0.1, 0.1, 100),
    y=[stock_loss(-0.02, pred) for pred in np.linspace(-0.1, 0.1, 100)],
    name='True value -0.02'
)

layout = go.Layout(
    xaxis=dict(
        title='prediction'
    ),
    yaxis=dict(
        title='loss'
    ),
    shapes=[{
            'type': 'line',
            'x0': 0,
            'y0': 0,
            'x1': 0,
            'y1': 1.5
        }]
)

fig = go.Figure(data=[trace_pos, trace_neg], layout=layout)

iplot(fig)

## Generate artificial data

Generate artificial data that is normally distributed around a straight line.

In [ ]:
N = 100

true_params = {
    'true_slope': 0.5,
    'true_intercept': 0.0
}

X = np.random.uniform(-0.06, 0.06, N)
Y = true_params['true_slope']*X + true_params['true_intercept'] + 0.01*np.random.randn(N)

In [ ]:
trace_data = go.Scatter(
    x=X,
    y=Y,
    mode='markers',
    name='data'
)

trace_true = go.Scatter(
    x=X,
    y=true_params['true_slope']*X + true_params['true_intercept'],
    name='true curve'
)

layout = go.Layout(
    xaxis=dict(
        title='signal'
    ),
    yaxis=dict(
        title='return'
    )
)

fig = go.Figure(data=[trace_data, trace_true], layout=layout)

iplot(fig)

## Modelling

Let's fit a model to the data with the form
$$
R = a + bx + \epsilon,
$$

where $a$ and $b$ are the unknown parameters and $\epsilon \sim \text{Normal}(0, \sigma)$ is normally-distributed random noise with unknown parameter $\sigma$.

We will use a Markov Chain Monte Carlo method to generate samples from the posterior distributions of the parameters $a, b, \sigma$, specifying the following priors for them
$$
\begin{array}{l}
a \sim \text{Normal}(0, 100),\\
b \sim \text{Normal}(0, 100),\\
\sigma \sim \text{Uniform}(0, 100)\\
\end{array}
$$

The model can be rewritten as a normal distribution centered on $a + bx$ with standard deviation $\sigma$, so we have

$$
R \sim \text{Normal}(\mu, \sigma),
$$

with $\mu = a + bx$.

In [ ]:
with pm.Model() as model:
    a = pm.Normal("a", mu=0, sd=100.)
    b = pm.Normal("b", mu=0, sd=100.)
    
    sigma = pm.Uniform("sigma", 0, 100)
    
    mu = pm.Deterministic("mu", a + b*X)
    
    obs = pm.Normal("obs", mu=mu, sd=sigma, observed=Y)
    
    trace = pm.sample(100000, step=pm.Metropolis())

We discard the first 20000 samples from the posterior distributions, as the Markov Chain might not have converged yet.

In [ ]:
burned_trace = trace[20000:]

In [ ]:
pm.plots.traceplot(trace=burned_trace, varnames=["a", "b", "sigma"])
pm.plot_posterior(trace=burned_trace, varnames=["a", "b", "sigma"], kde_plot=True)

For any particular value $x$ of the signal we can now compute a full distribution for the return given by our model evaluated with all the different values drawn from the posterior distributions or the parameters it depends upon.

Each element of the trace the MCMC returned is of the form $(a_i, b_i, \sigma_i)$, where the index $i$ runs over all the sampled values. Given the value $x$ then we have the whole population of returns given by
$$
R_i = a_i + b_i x + \epsilon,
$$

where $\epsilon$ is a sample from the distribution $\text{Normal}(0, \sigma_i)$.

## Predicting the optimal returns

Given a value $x$ for the signal, we can make a prediction $r$ for the return, and given this we can compute the expectation value of the loss function over the distribution of "true returns" associated with $x$,
$$
\mathbb{E}_{R(x)} \Bigl[ L(R(x), r) \Bigr].
$$

From a Bayesian point of view the optimal prediction is the one that minimises the above expectation value,
$$
r_\text{opt} = \text{argmin}_r\,\mathbb{E}_{R(x)} \Bigl[ L(R(x), r) \Bigr].
$$

The minimum of this function can be computed numerically.

In [ ]:
def vectorised_stock_loss(true_return, prediction, alpha=500.):
    """
    Loss function, vectorised.
    """
    loss = np.zeros_like(true_return)
    ix = true_return*prediction < 0
    loss[ix] = alpha*prediction**2 - np.sign(true_return[ix])*prediction + np.abs(true_return[ix])
    loss[~ix] = np.abs(true_return[~ix] - prediction)
    
    return loss

In [ ]:
a_samples = burned_trace['a']
b_samples = burned_trace['b']
sigma_trace = burned_trace['sigma']

num_samples = a_samples.shape[0]

In [ ]:
# Generator to sample the distribution for the random noise.
noise_generator = pm.Normal.dist(mu=0., sd=sigma_trace)

noise = noise_generator.random()

In [ ]:
returns_population = lambda signal: a_samples + b_samples*signal + noise

Let's compute the optimal prediction for 50 values of the signal.

In [ ]:
signals = np.linspace(X.min(), X.max(), 50)

opt_predictions = np.zeros_like(signals)

In [ ]:
for i, signal in tqdm(enumerate(signals)):
    exp_value = lambda pred: vectorised_stock_loss(returns_population(signal), pred).mean()
    
    opt_predictions[i] = fmin(exp_value, 0, disp=False)

Let's also compute the maximum likelihood predictions (a simple linear regression) for comparison.

In [ ]:
lr = LinearRegression()

lr.fit(X.reshape(-1,1), Y)

mle_pred = lr.predict(signals.reshape(-1, 1))

Compare the Bayesian optimal prediction with the maximum likelihood one: as a first check we can observe that for very clear (positive or negative signals) the Bayesian optimal prediction converges to the MLE one. The new feature though is that when the signal is not that certain, meaning that the outcome could be positive or negative, the Bayesian optimal prediction gets closer to 0 than the MLE one, essentially suggesting not to take a position on the outcome: the loss function is telling us that the signal is too uncertain to make a reliable prediction, thus the most optimal strategy to minimise (the expectation value of) loss, is to predict 0 and wait for clearer signals!

In [ ]:
trace_bayes_pred = go.Scatter(
    x=signals,
    y=opt_predictions,
    name='Bayesian optimal predictions'
)

trace_mle_pred = go.Scatter(
    x=signals,
    y=mle_pred,
    name='MLE predictions'
)

layout = go.Layout(
    xaxis=dict(
        title='signal'
    ),
    yaxis=dict(
        title='prediction'
    )
)

fig = go.Figure(data=[trace_bayes_pred, trace_mle_pred], layout=layout)

iplot(fig)